# Cell 1 - Import & Data loading

In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

DATA_PROCESSED_DIR = os.path.join("..", "data", "processed")
print("DATA_PROCESSED_DIR:", DATA_PROCESSED_DIR)

DATA_PROCESSED_DIR: ../data/processed


In [2]:
data_path = os.path.join(DATA_PROCESSED_DIR, "spotify_dataset_clustered.csv")

df = pd.read_csv(data_path)
print("Dataset loaded:", df.shape)
df.head()

Dataset loaded: (169909, 18)


,track_id,track_name,artist_name,popularity,year,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,duration_ms,pca_x,pca_y,cluster
0,6KbQ3uYMLKb5jDxLF7wYDD,Singende Bataillone 1. Teil,['Carl Woitschach'],0,1928,0.995,0.708,0.1950,0.563,0.1510,-12.428,0.0506,118.469,0.7790,158648,-1.010631,1.593194,7
1,6KuQTIu1KoTTkLXKrwlLPV,"Fantasiestücke, Op. 111: Più tosto lento","['Robert Schumann', 'Vladimir Horowitz']",0,1928,0.994,0.379,0.0135,0.901,0.0763,-28.454,0.0462,83.972,0.0767,282133,-4.751081,-0.113671,0
2,6L63VW0PibdM1HDSBoqnoM,Chapter 1.18 - Zamek kaniowski,['Seweryn Goszczyński'],0,1928,0.604,0.749,0.2200,0.000,0.1190,-19.924,0.9290,107.177,0.8800,104300,-0.184709,4.573615,7
3,6M94FkXd15sOAOQYRnWPN8,Bebamos Juntos - Instrumental (Remasterizado),['Francisco Canaro'],0,1928,0.995,0.781,0.1300,0.887,0.1110,-14.734,0.0926,108.003,0.7200,180760,-1.671672,1.857104,5
4,6N6tiFZ9vLTSOIxkj8qKrd,"Polonaise-Fantaisie in A-Flat Major, Op. 61","['Frédéric Chopin', 'Vladimir Horowitz']",1,1928,0.990,0.210,0.2040,0.908,0.0980,-16.829,0.0424,62.149,0.0693,687733,-4.018761,-2.630070,0


# Cell 2 - Defining audio for features for Fine Matching

In [3]:
feature_cols = [
    "acousticness", "danceability", "energy", "instrumentalness",
    "liveness", "loudness", "speechiness", "tempo", "valence", "duration_ms"
]

missing = [c for c in feature_cols if c not in df.columns]
print("Missing feature columns:", missing)


Missing feature columns: []


# Cell 3 - Clusters' Mapping (mood labels)

In [4]:
cluster_mood_labels = {
    0: "deep_focus_sad",          # alta acousticness, alta instrumentalness, bassa energy/valence
    1: "happy_party",             # alta danceability/energy/valence, loud
    2: "melancholic_acoustic",    # acoustic/ballad malinconiche
    3: "intense_rock",            # energy e loudness altissime, rock/metal
    4: "balanced_pop",            # pop “medio”, easy listening
    5: "calm_relax",              # calm/acoustic/relax
    6: "high_energy_workout",     # bangers EDM/workout
    7: "chill_rap_lofi"           # alto speechiness, chill rap/lofi
}

df["mood"] = df["cluster"].map(cluster_mood_labels)
df["mood"].value_counts()


mood
balanced_pop            30856
high_energy_workout     30008
happy_party             26795
calm_relax              20248
intense_rock            19966
melancholic_acoustic    16255
deep_focus_sad          13580
chill_rap_lofi          12201
Name: count, dtype: int64

# Cell 4 - Fitting StandardScaler globally + df_scaled

In [5]:
# Drop righe con NaN nelle feature
df_features = df.dropna(subset=feature_cols).copy()

scaler_global = StandardScaler()
features_scaled = scaler_global.fit_transform(df_features[feature_cols])

df_scaled = df_features.copy()
for i, col in enumerate(feature_cols):
    df_scaled[col + "_scaled"] = features_scaled[:, i]

print("Scaled dataframe shape:", df_scaled.shape)
df_scaled.head()


Scaled dataframe shape: (169909, 29)


,track_id,track_name,artist_name,popularity,year,acousticness,danceability,energy,instrumentalness,liveness,...,acousticness_scaled,danceability_scaled,energy_scaled,instrumentalness_scaled,liveness_scaled,loudness_scaled,speechiness_scaled,tempo_scaled,valence_scaled,duration_ms_scaled
0,6KbQ3uYMLKb5jDxLF7wYDD,Singende Bataillone 1. Teil,['Carl Woitschach'],0,1928,0.995,0.708,0.1950,0.563,0.1510,...,1.332319,0.968662,-1.097999,1.296562,-0.314998,-0.186652,-0.289840,0.049500,0.940924,-0.599713
1,6KuQTIu1KoTTkLXKrwlLPV,"Fantasiestücke, Op. 111: Più tosto lento","['Robert Schumann', 'Vladimir Horowitz']",0,1928,0.994,0.379,0.0135,0.901,0.0763,...,1.329664,-0.907636,-1.776785,2.389253,-0.737519,-3.014729,-0.319186,-1.073199,-1.735454,0.418119
2,6L63VW0PibdM1HDSBoqnoM,Chapter 1.18 - Zamek kaniowski,['Seweryn Goszczyński'],0,1928,0.604,0.749,0.2200,0.000,0.1190,...,0.294154,1.202486,-1.004503,-0.523513,-0.495997,-1.509457,5.568626,-0.317996,1.325822,-1.047680
3,6M94FkXd15sOAOQYRnWPN8,Bebamos Juntos - Instrumental (Remasterizado),['Francisco Canaro'],0,1928,0.995,0.781,0.1300,0.887,0.1110,...,1.332319,1.384983,-1.341091,2.343994,-0.541247,-0.593587,-0.009722,-0.291114,0.716082,-0.417454
4,6N6tiFZ9vLTSOIxkj8qKrd,"Polonaise-Fantaisie in A-Flat Major, Op. 61","['Frédéric Chopin', 'Vladimir Horowitz']",1,1928,0.990,0.210,0.2040,0.908,0.0980,...,1.319044,-1.871449,-1.064341,2.411883,-0.614778,-0.963288,-0.344530,-1.783425,-1.763655,3.761300


# Cell 5 - Mapping possible scenarios

In [6]:
mood_to_clusters = {
    "focus":      [0, 5],
    "sad":        [0, 2],
    "relax":      [0, 5, 7],
    "happy":      [1, 4, 7],
    "angry":      [3, 6],
    "party":      [1, 6],
    "workout":    [3, 6],
    "chill":      [5, 7],
}

activity_to_clusters = {
    "study":      [0, 5],
    "work":       [0, 4, 5],
    "workout":    [3, 6],
    "commute":    [1, 4, 7],
    "party":      [1, 6],
    "relax":      [0, 5, 7],
    "sleep":      [0, 5],
}

time_to_clusters = {
    "morning":    [4, 5],
    "afternoon":  [1, 4, 7],
    "evening":    [1, 3, 6, 7],
    "night":      [0, 2, 5, 7],
}

weather_to_clusters = {
    "sunny":      [1, 4, 6],
    "rain":       [0, 2, 5],
    "cloudy":     [2, 4, 5, 7],
    "snow":       [0, 2, 5],
}


# Cell 6 - Def(from context to cluster)

In [7]:
def get_clusters_for_context(
    user_mood=None,
    activity=None,
    time_of_day=None,
    weather=None
):
    clusters = set()

    if user_mood is not None:
        clusters.update(mood_to_clusters.get(user_mood, []))

    if activity is not None:
        clusters.update(activity_to_clusters.get(activity, []))

    if time_of_day is not None:
        clusters.update(time_to_clusters.get(time_of_day, []))

    if weather is not None:
        clusters.update(weather_to_clusters.get(weather, []))

    return sorted(list(clusters))

# test
get_clusters_for_context("relax", "study", "night", "rain")


[0, 2, 5, 7]

# Cell 7 - From age of the user, to get generation + formative years

In [8]:
DATASET_MIN_YEAR = int(df["year"].min())
DATASET_MAX_YEAR = int(df["year"].max())
print(DATASET_MIN_YEAR, DATASET_MAX_YEAR)

1921 2020


In [9]:
def get_user_generation(age):
    if age < 18:
        return "gen_z"
    elif 18 <= age < 30:
        return "gen_z_young_millennial"
    elif 30 <= age < 45:
        return "millennial"
    elif 45 <= age < 60:
        return "gen_x"
    else:
        return "boomer"


def get_formative_years(age, ref_year=DATASET_MAX_YEAR):
    """
    Formative years: [birth+15, birth+25], tagliati al range del dataset [1921, 2020].
    """
    birth_year = ref_year - age
    y1 = birth_year + 15
    y2 = birth_year + 25
    # clip nel range dataset
    y1 = max(y1, DATASET_MIN_YEAR)
    y2 = min(y2, DATASET_MAX_YEAR)
    if y1 > y2:
        # se succede (età assurda), fallback all'intero range
        y1, y2 = DATASET_MIN_YEAR, DATASET_MAX_YEAR
    return int(y1), int(y2)


In [10]:
mood_year_bias = {
    "sad":        (1995, DATASET_MAX_YEAR),
    "happy":      (2005, DATASET_MAX_YEAR),
    "relax":      (1960, DATASET_MAX_YEAR),
    "angry":      (1980, DATASET_MAX_YEAR),
    "party":      (2010, DATASET_MAX_YEAR),
    "chill":      (1990, DATASET_MAX_YEAR),
    "focus":      (1960, DATASET_MAX_YEAR),
}

activity_year_bias = {
    "study":      (1960, DATASET_MAX_YEAR),
    "work":       (1980, DATASET_MAX_YEAR),
    "workout":    (2010, DATASET_MAX_YEAR),
    "party":      (2010, DATASET_MAX_YEAR),
    "relax":      (1990, DATASET_MAX_YEAR),
    "sleep":      (1990, DATASET_MAX_YEAR),
    "commute":    (2000, DATASET_MAX_YEAR),
}


In [11]:
cluster_stats = (
    df.groupby("cluster")
      .agg(
          cluster_pop_mean=("popularity", "mean"),
          cluster_pop_std=("popularity", "std"),
          cluster_year_min=("year", "min"),
          cluster_year_max=("year", "max")
      )
)

cluster_stats


,cluster_pop_mean,cluster_pop_std,cluster_year_min,cluster_year_max
cluster,,,,
0,16.634831,19.478672,1921,2020
1,42.006419,17.542968,1921,2020
2,27.248847,19.976842,1921,2020
3,42.249875,14.169074,1921,2020
4,29.207674,20.234521,1921,2020
5,14.968985,17.642707,1921,2020
6,46.518162,14.819819,1921,2020
7,10.126793,16.017175,1921,2020


In [12]:
def build_target_profile(user_mood=None, activity=None, time_of_day=None, weather=None):
    """
    Restituisce un dict: feature -> valore target (nei range originali).
    Combina mood, activity, time_of_day, weather.
    """
    target = {
        "acousticness": 0.5,
        "danceability": 0.5,
        "energy": 0.5,
        "instrumentalness": 0.1,
        "liveness": 0.2,
        "loudness": -10.0,
        "speechiness": 0.1,
        "tempo": 120.0,
        "valence": 0.5,
        "duration_ms": 210000.0  # ~3.5 min
    }

    # --- Mood ---
    if user_mood == "sad":
        target["valence"] = 0.2
        target["energy"] = 0.3
        target["tempo"] = 95
        target["acousticness"] = 0.7
    elif user_mood == "happy":
        target["valence"] = 0.8
        target["energy"] = 0.7
        target["danceability"] = 0.7
    elif user_mood == "relax":
        target["energy"] = 0.3
        target["tempo"] = 90
        target["acousticness"] = 0.7
        target["valence"] = 0.5
    elif user_mood == "angry":
        target["energy"] = 0.9
        target["valence"] = 0.3
        target["tempo"] = 135
    elif user_mood == "party":
        target["energy"] = 0.8
        target["danceability"] = 0.8
        target["valence"] = 0.8
        target["tempo"] = 125
    elif user_mood == "chill":
        target["energy"] = 0.4
        target["danceability"] = 0.6
        target["tempo"] = 100
        target["valence"] = 0.6
    elif user_mood == "focus":
        target["energy"] = 0.3
        target["tempo"] = 90
        target["acousticness"] = 0.8
        target["speechiness"] = 0.05

    # --- Activity ---
    if activity == "study":
        target["energy"] = min(target["energy"], 0.4)
        target["acousticness"] = max(target["acousticness"], 0.6)
        target["speechiness"] = 0.1
    elif activity == "workout":
        target["energy"] = max(target["energy"], 0.8)
        target["tempo"] = max(target["tempo"], 125)
        target["danceability"] = max(target["danceability"], 0.7)
    elif activity == "relax":
        target["energy"] = min(target["energy"], 0.4)
        target["tempo"] = min(target["tempo"], 100)
    elif activity == "party":
        target["energy"] = max(target["energy"], 0.8)
        target["danceability"] = max(target["danceability"], 0.8)
        target["valence"] = max(target["valence"], 0.7)

    # --- Time of day ---
    if time_of_day == "night":
        target["energy"] = min(target["energy"], 0.5)
        target["tempo"] = min(target["tempo"], 105)
    elif time_of_day == "morning":
        target["energy"] = 0.5
        target["valence"] = max(target["valence"], 0.6)

    # --- Weather ---
    if weather == "rain":
        target["valence"] = min(target["valence"], 0.5)
        target["acousticness"] = max(target["acousticness"], 0.6)
    elif weather == "sunny":
        target["valence"] = max(target["valence"], 0.7)

    return target

# esempio
build_target_profile("sad", "study", "night", "rain")


{'acousticness': 0.7,
 'danceability': 0.5,
 'energy': 0.3,
 'instrumentalness': 0.1,
 'liveness': 0.2,
 'loudness': -10.0,
 'speechiness': 0.1,
 'tempo': 95,
 'valence': 0.2,
 'duration_ms': 210000.0}

In [13]:
def rank_by_target_profile(subset, target_profile, top_n=10):
    """
    subset: DataFrame già filtrato (per cluster / anno / popolarità).
            Deve avere indice compatibile con df_scaled.
    """
    if subset.empty:
        return subset

    # Intersezione degli indici con df_scaled (feature complete)
    valid_index = subset.index.intersection(df_scaled.index)
    subset = subset.loc[valid_index]

    if subset.empty:
        return subset

    # Vettore target raw
    target_vec_raw = np.array([target_profile[col] for col in feature_cols]).reshape(1, -1)
    # spazio scalato
    target_vec_scaled = scaler_global.transform(target_vec_raw)[0]

    scaled_cols = [col + "_scaled" for col in feature_cols]
    subset_scaled = df_scaled.loc[subset.index, scaled_cols].values

    # Pesi (puoi modificarli)
    weights = np.array([
        0.8,  # acousticness
        1.0,  # danceability
        1.2,  # energy
        0.5,  # instrumentalness
        0.5,  # liveness
        1.0,  # loudness
        0.8,  # speechiness
        1.0,  # tempo
        1.2,  # valence
        0.7,  # duration_ms
    ])

    diff = subset_scaled - target_vec_scaled
    dists = np.sqrt(np.sum((diff * weights) ** 2, axis=1))

    subset_ranked = subset.copy()
    subset_ranked["distance_to_target"] = dists
    subset_ranked = subset_ranked.sort_values("distance_to_target", ascending=True)

    return subset_ranked.head(top_n)


In [14]:
def intersect_year_ranges(ranges):
    """
    ranges: lista di tuple (y1, y2) o None.
    Restituisce (low, high) come intersezione; se vuota → None.
    """
    lows = []
    highs = []
    for r in ranges:
        if r is None:
            continue
        low, high = r
        lows.append(low)
        highs.append(high)
    if not lows:
        return None
    low_final = max(lows)
    high_final = min(highs)
    if low_final > high_final:
        return None
    return int(low_final), int(high_final)


In [15]:
def recommend_tracks(
    df,
    n=10,
    user_mood=None,
    activity=None,
    time_of_day=None,
    weather=None,
    age=None,
    min_popularity=None,   # opzionale: override manuale
    year_from=None,        # opzionale: override manuale
    year_to=None,
):
    # 1) Cluster macro
    target_clusters = get_clusters_for_context(
        user_mood=user_mood,
        activity=activity,
        time_of_day=time_of_day,
        weather=weather
    )
    print("Target clusters:", target_clusters)
    if not target_clusters:
        subset = df.copy()
    else:
        subset = df[df["cluster"].isin(target_clusters)].copy()

    # 2) Costruisci range di anni da 4 fattori

    year_ranges = []

    # 2.1 Età -> formative years
    if age is not None:
        gen = get_user_generation(age)
        fy1, fy2 = get_formative_years(age)
        print(f"Età utente: {age} → generazione: {gen}, formative years: {fy1}-{fy2}")
        year_ranges.append((fy1, fy2))

    # 2.2 Mood bias
    if user_mood in mood_year_bias:
        year_ranges.append(mood_year_bias[user_mood])

    # 2.3 Activity bias
    if activity in activity_year_bias:
        year_ranges.append(activity_year_bias[activity])

    # 2.4 Cluster year range naturale (media dei cluster target)
    if target_clusters:
        cmin = cluster_stats.loc[target_clusters, "cluster_year_min"].min()
        cmax = cluster_stats.loc[target_clusters, "cluster_year_max"].max()
        year_ranges.append((cmin, cmax))

    # Intersezione
    auto_year_range = intersect_year_ranges(year_ranges)
    if auto_year_range is not None:
        y1_auto, y2_auto = auto_year_range
        print(f"Year range auto (da età+mood+activity+cluster): {y1_auto}-{y2_auto}")
        if "year" in subset.columns:
            subset = subset[
                (subset["year"].fillna(0) >= y1_auto) &
                (subset["year"].fillna(0) <= y2_auto)
            ]

    # 3) Popularità dinamica basata su cluster e generazione
    auto_min_pop = 0
    if target_clusters:
        mean_cluster_pop = cluster_stats.loc[target_clusters, "cluster_pop_mean"].mean()
        if mean_cluster_pop >= 60:
            auto_min_pop = 40
        elif mean_cluster_pop >= 40:
            auto_min_pop = 30
        else:
            auto_min_pop = 20

    if age is not None:
        gen = get_user_generation(age)
        gen_boost = {
            "gen_z": 10,
            "gen_z_young_millennial": 8,
            "millennial": 5,
            "gen_x": 3,
            "boomer": 0
        }[gen]
        auto_min_pop += gen_boost

    if "popularity" in subset.columns and auto_min_pop > 0:
        print(f"Popularity threshold auto: {auto_min_pop}")
        subset = subset[subset["popularity"].fillna(0) >= auto_min_pop]

    # 4) Override manuale (se user specifica)
    if min_popularity is not None and "popularity" in subset.columns:
        print(f"Override manuale min_popularity: {min_popularity}")
        subset = subset[subset["popularity"].fillna(0) >= min_popularity]

    if "year" in subset.columns:
        if year_from is not None:
            subset = subset[subset["year"] >= year_from]
        if year_to is not None:
            subset = subset[subset["year"] <= year_to]

    if subset.empty:
        print("⚠️ Nessun brano trovato dopo i filtri → fallback al dataset completo.")
        subset = df.copy()

    # 5) Profilo target (feature)
    target_profile = build_target_profile(
        user_mood=user_mood,
        activity=activity,
        time_of_day=time_of_day,
        weather=weather
    )

    # 6) Ranking per distanza (sotto-cluster continuo)
    ranked = rank_by_target_profile(subset, target_profile, top_n=n)

    if ranked.empty:
        print("⚠️ Nessun brano con tutte le feature → sample casuale.")
        ranked = subset.sample(min(n, len(subset)))

    cols = [
        c for c in [
            "track_name", "artist_name", "year", "mood", "cluster",
            "popularity", "distance_to_target"
        ] if c in ranked.columns
    ]
    return ranked[cols].reset_index(drop=True)


In [16]:
def explain_context(
    user_mood=None,
    activity=None,
    time_of_day=None,
    weather=None,
    age=None
):
    clusters = get_clusters_for_context(user_mood, activity, time_of_day, weather)
    moods = [cluster_mood_labels[c] for c in clusters]

    print("Context:")
    print("  mood      :", user_mood)
    print("  activity  :", activity)
    print("  time_of_day:", time_of_day)
    print("  weather   :", weather)
    print("  age       :", age)
    print("\nClusters selezionati:", clusters)
    print("Mood descrittivi:", moods)

    if age is not None:
        gen = get_user_generation(age)
        fy1, fy2 = get_formative_years(age)
        print(f"\nGenerazione stimata: {gen}, formative years: {fy1}-{fy2}")

    target_profile = build_target_profile(user_mood, activity, time_of_day, weather)
    print("\nTarget profile (feature):")
    for k, v in target_profile.items():
        print(f"  {k:15s} -> {v}")


# EXAMPLES

In [17]:
explain_context("sad", "study", "night", "rain", age=22)

recs1 = recommend_tracks(
    df,
    n=10,
    user_mood="sad",
    activity="study",
    time_of_day="night",
    weather="rain",
    age=22
)
recs1

Context:
  mood      : sad
  activity  : study
  time_of_day: night
  weather   : rain
  age       : 22

Clusters selezionati: [0, 2, 5, 7]
Mood descrittivi: ['deep_focus_sad', 'melancholic_acoustic', 'calm_relax', 'chill_rap_lofi']

Generazione stimata: gen_z_young_millennial, formative years: 2013-2020

Target profile (feature):
  acousticness    -> 0.7
  danceability    -> 0.5
  energy          -> 0.3
  instrumentalness -> 0.1
  liveness        -> 0.2
  loudness        -> -10.0
  speechiness     -> 0.1
  tempo           -> 95
  valence         -> 0.2
  duration_ms     -> 210000.0
Target clusters: [0, 2, 5, 7]
Età utente: 22 → generazione: gen_z_young_millennial, formative years: 2013-2020
Year range auto (da età+mood+activity+cluster): 2013-2020
Popularity threshold auto: 28


/Users/Vitaliano/Desktop/Data Science/FDS - Fundamentals of Data Science/Homework di Gruppo/Final Project/Music-mood pred/.venv/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


,track_name,artist_name,year,mood,cluster,popularity,distance_to_target
0,Lose It,['Oh Wonder'],2015,melancholic_acoustic,2,64,0.638747
1,Mama's Gun,['Glass Animals'],2016,melancholic_acoustic,2,55,0.650693
2,"If You Need To, Keep Time on Me",['Fleet Foxes'],2017,melancholic_acoustic,2,64,0.701045
3,Open,['Rhye'],2013,melancholic_acoustic,2,64,0.720574
4,Cupid,['Ryan Beatty'],2018,melancholic_acoustic,2,59,0.726374
5,O God Forgive Us,"['for KING & COUNTRY', 'KB']",2015,melancholic_acoustic,2,51,0.732798
6,Every Kind Of Way,['H.E.R.'],2017,melancholic_acoustic,2,65,0.736861
7,Drift,"['Galimatias', 'Alina Baraz']",2015,melancholic_acoustic,2,50,0.741993
8,Lose You To Love Me,['Selena Gomez'],2020,melancholic_acoustic,2,85,0.754090
9,Black and White,['Rainbow Kitten Surprise'],2013,calm_relax,5,48,0.759292


In [20]:
explain_context("focus", "study", "morning", "sunny", age=23)

recs3 = recommend_tracks(
    df,
    n=10,
    user_mood="focus",
    activity="study",
    time_of_day="morning",
    weather="sunny",
    age=23
)
recs3


Context:
  mood      : focus
  activity  : study
  time_of_day: morning
  weather   : sunny
  age       : 23

Clusters selezionati: [0, 1, 4, 5, 6]
Mood descrittivi: ['deep_focus_sad', 'happy_party', 'balanced_pop', 'calm_relax', 'high_energy_workout']

Generazione stimata: gen_z_young_millennial, formative years: 2012-2020

Target profile (feature):
  acousticness    -> 0.8
  danceability    -> 0.5
  energy          -> 0.5
  instrumentalness -> 0.1
  liveness        -> 0.2
  loudness        -> -10.0
  speechiness     -> 0.1
  tempo           -> 90
  valence         -> 0.7
  duration_ms     -> 210000.0
Target clusters: [0, 1, 4, 5, 6]
Età utente: 23 → generazione: gen_z_young_millennial, formative years: 2012-2020
Year range auto (da età+mood+activity+cluster): 2012-2020
Popularity threshold auto: 28


/Users/Vitaliano/Desktop/Data Science/FDS - Fundamentals of Data Science/Homework di Gruppo/Final Project/Music-mood pred/.venv/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


,track_name,artist_name,year,mood,cluster,popularity,distance_to_target
0,Call It Dreaming,['Iron & Wine'],2017,balanced_pop,4,63,0.733743
1,Call It Dreaming,['Iron & Wine'],2017,balanced_pop,4,63,0.733743
2,Intro,['Logic'],2014,balanced_pop,4,51,0.741194
3,"Love, Love, Love",['Donny Hathaway'],2013,balanced_pop,4,51,0.747676
4,If the World Was Ending - feat. Julia Michaels,"['JP Saxe', 'Julia Michaels']",2019,balanced_pop,4,89,0.774178
5,No Gray,['Jonathan McReynolds'],2012,balanced_pop,4,48,0.858671
6,Out in the Country,['Natural Child'],2014,balanced_pop,4,50,0.900598
7,Brothers,['Lil Tjay'],2019,balanced_pop,4,63,0.978339
8,Olivia,['Rayland Baxter'],2012,balanced_pop,4,61,1.002893
9,Sing About It,['The Wood Brothers'],2013,balanced_pop,4,56,1.026815
